In [1]:
import os
from dotenv import load_dotenv

from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_openai import OpenAIEmbeddings
from langchain_ollama.embeddings import OllamaEmbeddings

from langchain.vectorstores import Chroma
from langchain.vectorstores import FAISS

from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

In [10]:
load_dotenv()

True

In [11]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [2]:
loder = PyPDFLoader("./data/law.pdf")

docs = loder.load()

In [3]:
len(docs)

164

In [4]:
print(docs[80].page_content)

shall be punished with imprisonment of either description for a term which may extend 
to three years, and shall also be liable to fine.  
  
(2) Any person of the Qaudiani group or Lahori group (who call themselves "Ahmadis" or 
by any other name) who by words, either spoken or written, or by visible 
representation refers to the mode or form of call to prayers followed by his faith as 
"Azan", or recites Azan as used by the Muslims, shall be punished with imprisonment 
of either description for a term which may extend to three years, and shall also be 
liable to fine.  
  
 
  
298-
C. 
Person of Quadiani group, etc., calling himself a Muslim or preaching or propagating 
his faith: 
Any person of the Quadiani group or the Lahori group (who call themselves 'Ahmadis' or 
by any other name), who directly or indirectly, poses himself as a Muslim, or calls, or 
refers to, his faith as Islam, or preaches or propagates his faith, or invites others to accept 
his faith, by words, either spok

In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=300)
text = text_splitter.split_documents(docs)

In [6]:
len(text)

392

In [7]:
print(text[60].page_content)

abetment, be punished with the punishment provided for the offence:  
 26[  
Provided that, except in case of Ikrah-i-Tam, the, abettor of an offence referred to in 
Chapter XVI shall be liable to punishment of ta'zir specified for such offence including 
death.  
] 26 
Explanation: An act or offence is said-to be committed in consequence of abetment, when 
it is committed in consequence of the instigation, or in pursuance of the conspiracy, or with 
the aid which constitutes the abetment.  
 
Illustrations 
(a) A offers a bribe to B, a public servant, as a reward for showing A some favour in the 
exercise of B's official functions. B accepts the bribe. A has abetted the offence defined 
in Section 161.  
  
(b) A instigates B to give false evidence. B, in consequence of the instigation commits that 
offence. A is guilty of abetting that offence, and is liable to the same punishment as B. 
  
(c) A and B conspire to poison Z. A, in pursuance of the conspiracy, procures the poison 
and 

In [13]:
# emdeddings = OllamaEmbeddings(model="nomic-embed-text")
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [17]:
faiss_vector_store = FAISS.from_documents(documents=text, embedding=embeddings)

In [18]:
faiss_vector_store.save_local("faiss_index")

In [56]:
retriever = faiss_vector_store.as_retriever(kwargs={"search_kwargs": {"k": 5}})

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant and your name is 'Jolly LLB' that answers questions about law in saimple and understandable language based on the given context only. if the context does not have enough information then say I'don;t have information abouth this question. Context is: {context}."),
    ("human", "{input}"),
])
prompt

ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='You are a helpful assistant that answers questions about law in saimple and understandable language based on the given {context} only.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])

In [48]:
llm = ChatGroq(model="deepseek-r1-distill-llama-70b")

In [ ]:
document_chain = create_stuff_documents_chain(llm, prompt)

In [57]:
retriever_chain  = create_retrieval_chain(retriever=retriever, combine_docs_chain=document_chain)

In [58]:
result = retriever_chain.invoke({"input":"What is the punishment for murder?"})
result

{'input': 'What is the punishment for murder?',
 'context': [Document(id='4e5707f6-6094-4ed0-8857-fbeb612d60fc', metadata={'producer': 'Acrobat Distiller 6.0 (Windows)', 'creator': 'PScript5.dll Version 5.2', 'creationdate': '2002-01-02T19:58:15+05:00', 'moddate': '2002-01-02T19:58:15+05:00', 'title': 'Microsoft Word - Pakistan Penal Code.doc', 'author': 'Administrator', 'source': './data/law.pdf', 'total_pages': 164, 'page': 27, 'page_label': '28'}, page_content='murder. \n  \n114. Abettor present when offence is committed: \nWhenever any person, who if absent would be liable to be punished as an abettor, is present \nwhen the act or offence for which he would be punishable in consequence of the abetment is \ncommitted, he shall be deemed to have committed such act or offence. \n  \n115. Abetment of offence punishable with death or imprisonment for life if offence not \ncommitted: \nWhoever abets the commission of an offence punishable with death or imprisonment for \nlife, shall, if 

In [59]:
print(result["answer"])

<think>
Okay, so I need to figure out the punishment for murder based on the given legal sections. Let me start by going through each section to understand how they relate to murder and its punishment.

First, I see sections about abettors and their liability. Section 114 talks about an abettor being present when the offense is committed, making them liable as if they committed it. Section 115 deals with abetting offenses punishable by death or life imprisonment. If the offense isn't committed, the abettor can get up to seven years in prison. If someone is hurt, it can go up to fourteen years. So, if someone abets murder but the murder isn't committed, they can still be punished, but not as severely as if the murder happened.

Looking at sections 364 and 365, these are about kidnapping or abducting someone under fourteen with the intent to murder or cause harm. The punishment here can be death, life imprisonment, or up to fourteen years in jail. So, if someone kidnaps a child with the 

In [60]:
print(result["context"][1].page_content)

be so disposed of as to be put in danger of being murdered, shall be punished with 
imprisonment for life or rigorous imprisonment for a term which may extend to ten years 
and shall also be liable to fine.  
Illustrations 
(a) A kidnaps Z from Pakistan, intending or knowing it to be likely that Z may be 
sacrificed to an idol. A has committed the offence defined in this section.  
  
(b) A forcibly carries or entices 5 away from his home in order that B may be murdered. A 
has committed the offence defined in this section.  
  
 
  
364-
A. 
Kidnapping or abducting a person under the 140[age of fourteen] 140: 
Whoever kidnaps or abducts any person under the  141[age of fourteen] 141 in order that such 
person may be murdered or subjected to grievous hurt, or slavery, or to the lust of any 
person or may be so disposed of as to be put in danger of being murdered or subjected to 
grievous hurt, or slavery, or to the lust of any person shall be punished with death or with 
imprisonment f

In [65]:
result = retriever_chain.invoke({"input":"Responsibilities of the PM?"})
result

{'input': 'Responsibilities of the PM?',
 'context': [Document(id='23fd5fd5-7520-49ce-93ee-da9c39039491', metadata={'producer': 'Acrobat Distiller 6.0 (Windows)', 'creator': 'PScript5.dll Version 5.2', 'creationdate': '2002-01-02T19:58:15+05:00', 'moddate': '2002-01-02T19:58:15+05:00', 'title': 'Microsoft Word - Pakistan Penal Code.doc', 'author': 'Administrator', 'source': './data/law.pdf', 'total_pages': 164, 'page': 3, 'page_label': '4'}, page_content='of such duties;  \nFifth:  Every juryman, assessor, or member of  a panchayat assisting a Court of Justice or \npublic servant;  \nSixth:  Every arbitrator or other person to whom any cause or matter has been referred for \ndecision or report by any Court of Justice, or by any other competent public \nauthority;  \nSeventh:  Every person who holds any office by virtue of which he is empowered to place \nor keep any person in confinement;  \nEighth:  Every officer of the Government wh ose duty it is, as such officer, to prevent \noffen

In [66]:
print(result["answer"])

<think>
Okay, so I'm trying to understand the responsibilities of the Prime Minister (PM) in Pakistan based on the provided legal text. The text is a bit dense and filled with legal jargon, so I'll need to break it down step by step.

First, I notice that the text lists several categories of people who fall under certain duties and responsibilities, especially related to public service. These include jury members, assessors, arbitrators, and various government officers. The PM is mentioned in the ninth point under "Every officer in executive Government in Pakistan," which suggests that the PM is part of the executive branch and has specific duties tied to their role.

Looking further, the text discusses election expenses and the failure to keep proper accounts, which might relate to the PM's role in ensuring legal compliance, especially regarding financial transparency. There's also mention of assisting public servants and the consequences of failing to do so, which could imply that th